# Make MTH5 from Metronix ATSS + JSON

Metronix Geophysics as developed a new folder structure and file system for their data that tries to mimic MTH5 making the translation relatively straight forward.  More information can be found at [Metronix Geophysics HDF5](https://mth5.geo-metronix.de/introduction/introduction.html).

The data comes in pairs for each channel of a run:

- `.atss` files, which are simply the data that can be easily read into a numpy array
- `.json` metadata file with the same file basename as the `.atss` file 

The data are then organized in a folder system that looks like:

```
Northern Mining
├── config
├── db
├── dump
├── edi
├── filters
├── jle
├── jobs
├── log
├── meta
│   └── Sarıçam
│       ├── run_001
│       │   ├── 084_2009-08-20_13-22-00_2009-08-21_07-00-00_R001_128H.xml
│       │   ├── 084_ADU-07e_C000_TEx_128Hz.json
│       │   ├── 084_ADU-07e_C001_TEy_128Hz.json
│       │   ├── 084_ADU-07e_C002_THx_128Hz.json
│       │   ├── 084_ADU-07e_C003_THy_128Hz.json
│       │   └── 084_ADU-07e_C004_THz_128Hz.json
│       └── run_002
│           ├── 084_2009-08-21_07-01-00_2009-08-21_07-06-00_R001_2048H.xml
│           ├── 084_ADU-07e_C000_TEx_2048Hz.json
│           ├── 084_ADU-07e_C001_TEy_2048Hz.json
│           ├── 084_ADU-07e_C002_THx_2048Hz.json
│           ├── 084_ADU-07e_C003_THy_2048Hz.json
│           └── 084_ADU-07e_C004_THz_2048Hz.json
├── processings
├── reports
│   └── old_cal
│       ├── FGS03E_000_master.txt
│       ├── MFS05141.TXT
│       ├── MFS06024.TXT
│       ├── MFS06026.TXT
│       ├── MFS06032.TXT
│       ├── MFS06E_000_master.txt
│       ├── MFS07E_000_master.txt
│       └── MFS10e0021.TXT
├── shell
│   ├── mkallproc.sh
│   ├── plot_ascii_table_edi.sh
│   └── procall.sh
├── stations
│   └── Sarıçam
│       ├── run_001
│       │   ├── 084_ADU-07e_C000_TEx_128Hz.atss
│       │   ├── 084_ADU-07e_C000_TEx_128Hz.json
│       │   ├── 084_ADU-07e_C001_TEy_128Hz.atss
│       │   ├── 084_ADU-07e_C001_TEy_128Hz.json
│       │   ├── 084_ADU-07e_C002_THx_128Hz.atss
│       │   ├── 084_ADU-07e_C002_THx_128Hz.json
│       │   ├── 084_ADU-07e_C003_THy_128Hz.atss
│       │   ├── 084_ADU-07e_C003_THy_128Hz.json
│       │   ├── 084_ADU-07e_C004_THz_128Hz.atss
│       │   └── 084_ADU-07e_C004_THz_128Hz.json
│       └── run_002
│           ├── 084_ADU-07e_C000_TEx_2048Hz.atss
│           ├── 084_ADU-07e_C000_TEx_2048Hz.json
│           ├── 084_ADU-07e_C001_TEy_2048Hz.atss
│           ├── 084_ADU-07e_C001_TEy_2048Hz.json
│           ├── 084_ADU-07e_C002_THx_2048Hz.atss
│           ├── 084_ADU-07e_C002_THx_2048Hz.json
│           ├── 084_ADU-07e_C003_THy_2048Hz.atss
│           ├── 084_ADU-07e_C003_THy_2048Hz.json
│           ├── 084_ADU-07e_C004_THz_2048Hz.atss
│           └── 084_ADU-07e_C004_THz_2048Hz.json
└── tmp
```

## Metadata

Metadata is basic, most of the important values are in the `.json` files.  ID's for run, station, survey are taken from the file structure and folder names.  

## Filters

This is still a work in progress, but the sensor response filters are in the `.json` files and are included in the MTH5.  What is still unclear is any instrument response that may need to be included.  

There is no filter for dipole length as the instrument calculates that internally and stores electric channels in mV/km.

## Example Data
You can find example data [here](https://cloud.geo-metronix.de/s/WCiDGispeeiQS9G). Download one of the folders, `small_example/Northern_Mining` is what is used in this example. Just change the file path to your local machine.

Here we are going to create a MTH5 from all the sample rates provided.  Note all sample rates are given in units of samples per second.

We are also going to keep the run names as they are in the folder structure.  If you want to change that behavior and name the run by sample rate you can change the keyword `run_name_zeros` to an integer.  By doing this the run ID will change from `run_001` to `sr{sample_rate}_{1:0{run_name_zeros}}`, for example a sample rate of 32 will become `sr32_001`.  


In [3]:
from mth5.clients import MakeMTH5
from mth5_test_data import get_test_data_path
from mth5.mth5 import MTH5

metronix_data_path = get_test_data_path("metronix")

In [4]:
metronix_mth5_path = MakeMTH5.from_metronix(
    metronix_data_path,
    sample_rates=[2048, 512, 32, 1./2],
    mth5_filename="from_metronix_test.h5",
    run_name_zeros=0)

2026-01-04T11:59:59.927677-0800 | WARNING | mth5.mth5 | open_mth5 | line: 610 | from_metronix_test.h5 will be overwritten in 'w' mode
2026-01-04T12:00:00.178871-0800 | INFO | mth5.mth5 | _initialize_file | line: 678 | Initialized MTH5 0.2.0 file C:\Users\peaco\OneDrive\Documents\GitHub\mth5_test_data\mth5_test_data\metronix\from_metronix_test.h5 in mode w
2026-01-04T12:00:07.845223-0800 | INFO | mth5.mth5 | close_mth5 | line: 772 | Flushing and closing C:\Users\peaco\OneDrive\Documents\GitHub\mth5_test_data\mth5_test_data\metronix\from_metronix_test.h5
2026-01-04T12:00:07.848327-0800 | INFO | mth5.clients.metronix | make_mth5_from_metronix | line: 125 | Wrote MTH5 file to: C:\Users\peaco\OneDrive\Documents\GitHub\mth5_test_data\mth5_test_data\metronix\from_metronix_test.h5


In [5]:
with MTH5() as m:
    m = m.open_mth5(metronix_mth5_path)
    print(m)
    channel_df = m.channel_summary.to_dataframe()
    run_df = m.run_summary

/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
            |- Group: Northern_Mining
            -------------------------
                |- Group: Filters
                -----------------
                    |- Group: coefficient
                    ---------------------
                    |- Group: fap
                    -------------
                        |- Group: mfs-06_chopper_0
                        --------------------------
                            --> Dataset: fap_table
                            ........................
                        |- Group: mfs-06_chopper_1
                        --------------------------
                            --> Dataset: fap_table
                            ........................
           

## Channel Summary

In [8]:

channel_df

,survey,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,has_data,hdf5_reference,run_hdf5_reference,station_hdf5_reference
0,Northern_Mining,saricam,run_002,39.026198,29.12395,1087.35,ex,2009-08-20 13:22:01+00:00,2009-08-21 06:59:59.718750+00:00,2031320,32.0,electric,0.0,0.0,milliVolt per kilometer,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
1,Northern_Mining,saricam,run_002,39.026198,29.12395,1087.35,ey,2009-08-20 13:22:01+00:00,2009-08-21 06:59:59.718750+00:00,2031320,32.0,electric,90.0,0.0,milliVolt per kilometer,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
2,Northern_Mining,saricam,run_002,39.026198,29.12395,1087.35,hx,2009-08-20 13:22:01+00:00,2009-08-21 06:59:59.718750+00:00,2031320,32.0,magnetic,0.0,0.0,milliVolt,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
3,Northern_Mining,saricam,run_002,39.026198,29.12395,1087.35,hy,2009-08-20 13:22:01+00:00,2009-08-21 06:59:59.718750+00:00,2031320,32.0,magnetic,90.0,0.0,milliVolt,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4,Northern_Mining,saricam,run_002,39.026198,29.12395,1087.35,hz,2009-08-20 13:22:01+00:00,2009-08-21 06:59:59.718750+00:00,2031320,32.0,magnetic,0.0,90.0,milliVolt,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
5,Northern_Mining,saricam,run_005,39.026198,29.12395,1087.35,ex,2009-08-20 13:22:25+00:00,2009-08-21 06:59:35+00:00,31716,0.5,electric,0.0,0.0,milliVolt per kilometer,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
6,Northern_Mining,saricam,run_005,39.026198,29.12395,1087.35,ey,2009-08-20 13:22:25+00:00,2009-08-21 06:59:35+00:00,31716,0.5,electric,90.0,0.0,milliVolt per kilometer,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
7,Northern_Mining,saricam,run_005,39.026198,29.12395,1087.35,hx,2009-08-20 13:22:25+00:00,2009-08-21 06:59:35+00:00,31716,0.5,magnetic,0.0,0.0,milliVolt,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
8,Northern_Mining,saricam,run_005,39.026198,29.12395,1087.35,hy,2009-08-20 13:22:25+00:00,2009-08-21 06:59:35+00:00,31716,0.5,magnetic,90.0,0.0,milliVolt,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
9,Northern_Mining,saricam,run_005,39.026198,29.12395,1087.35,hz,2009-08-20 13:22:25+00:00,2009-08-21 06:59:35+00:00,31716,0.5,magnetic,0.0,90.0,milliVolt,True,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>


## Run Summary

In [7]:
run_df

,channel_scale_factors,duration,end,has_data,input_channels,mth5_path,n_samples,output_channels,run,sample_rate,start,station,survey,run_hdf5_reference,station_hdf5_reference
0,"{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",63478.718750,2009-08-21 06:59:59.718750+00:00,True,"[hx, hy]",C:/Users/peaco/OneDrive/Documents/GitHub/mth5_...,2031320,"[ex, ey, hz]",run_002,32.0,2009-08-20 13:22:01+00:00,saricam,Northern_Mining,<HDF5 object reference>,<HDF5 object reference>
1,"{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",63430.000000,2009-08-21 06:59:35+00:00,True,"[hx, hy]",C:/Users/peaco/OneDrive/Documents/GitHub/mth5_...,31716,"[ex, ey, hz]",run_005,0.5,2009-08-20 13:22:25+00:00,saricam,Northern_Mining,<HDF5 object reference>,<HDF5 object reference>
2,"{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",299.999512,2009-08-21 07:05:59.999512+00:00,True,"[hx, hy]",C:/Users/peaco/OneDrive/Documents/GitHub/mth5_...,614400,"[ex, ey, hz]",run_007,2048.0,2009-08-21 07:01:00+00:00,saricam,Northern_Mining,<HDF5 object reference>,<HDF5 object reference>
3,"{'ex': 1.0, 'ey': 1.0, 'hx': 1.0, 'hy': 1.0, '...",298.982422,2009-08-21 07:05:59.982422+00:00,True,"[hx, hy]",C:/Users/peaco/OneDrive/Documents/GitHub/mth5_...,153080,"[ex, ey, hz]",run_008,512.0,2009-08-21 07:01:01+00:00,saricam,Northern_Mining,<HDF5 object reference>,<HDF5 object reference>
